In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [2]:
# Opciones de Chrome para hacer webscraping
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920x1080')   # Ejecutar en segundo plano
options.add_argument('--incognito') # Modo incógnito

In [1]:
#Ejemplo para 10 atletas
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

resultados = []

# Página 1 solamente
url = "https://results.baa.org/2024/?event_main_group=runner&num_results=1000&pid=search&page=1&search%5Bage_class%5D=%25&search%5Bnation%5D=%25&search_sort=time_finish_netto&event=R"
driver.get(url)
time.sleep(4)

# Solo los primeros 10 atletas
atletas = driver.find_elements(By.CSS_SELECTOR, "li.list-group-item.row")[:10]

for index, atleta in enumerate(atletas):
    try:
        place_overall = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-place.place-secondary.hidden-xs.numeric").text
    except:
        place_overall = "Sin posición general"

    try:
        place_gender = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-place.place-primary.numeric").text
    except:
        place_gender = "Sin posición género"

    try:
        nombre = atleta.find_element(By.CLASS_NAME, "list-field.type-fullname").text
    except:
        nombre = "Sin nombre"

    try:
        finish_net_label = atleta.find_element(By.XPATH, ".//div[.='Finish Net']")
        tiempo_neto = finish_net_label.find_element(By.XPATH, "../following-sibling::div").text.strip()
    except:
        tiempo_neto = "Sin tiempo neto"

    # Ir al perfil individual y sacar división
    try:
        enlace = atleta.find_element(By.TAG_NAME, "a").get_attribute("href")
        driver.execute_script("window.open(arguments[0]);", enlace)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(1.5)

        try:
            division = driver.find_element(By.XPATH, "//td[contains(@class, 'f-_type_age_class')]").text.strip()
        except:
            division = "Sin división"

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    except:
        division = "Sin división"

    resultados.append({
        "place_overall": place_overall,
        "place_gender": place_gender,
        "nombre": nombre,
        "tiempo_neto": tiempo_neto,
        "division": division
    })

    time.sleep(0.3)

driver.quit()

In [3]:
resultados_df = pd.DataFrame(resultados)
resultados_df

,place_overall,place_gender,nombre,tiempo_neto,division
0,Sin posición general,Sin posición género,Name,Finish Gun,Sin división
1,1,1,"Lemma, Sisay",02:06:17,18-39
2,2,2,"Esa, Mohamed",02:06:58,18-39
3,3,3,"Chebet, Evans",02:07:22,18-39
4,4,4,"Korir, John",02:07:40,18-39
5,5,5,"Korir, Albert",02:07:47,18-39
6,6,6,"Mpofu, Isaac",02:08:17,18-39
7,,7,"Albertson, Cj",Finish Gun\n02:09:53,18-39
8,,8,"Morii, Yuma",Finish Gun\n02:09:59,18-39
9,,9,"Kotut, Cybrian",Finish Gun\n02:10:29,18-39


In [4]:
#guardar en CSV
resultados_df.to_csv("../data/ejemplo_10_resultados_boston.csv", index=False)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

resultados = []

for pagina in range(1, 2):  # Puedes cambiar el rango para hacer scraping por bloques
    print(f"\n📄 Página {pagina}...")
    url = f"https://results.baa.org/2024/?event_main_group=runner&num_results=1000&pid=search&page={pagina}&search%5Bage_class%5D=%25&search%5Bnation%5D=%25&search_sort=time_finish_netto&event=R"
    driver.get(url)
    time.sleep(4)

    atletas = driver.find_elements(By.CSS_SELECTOR, "li.list-group-item.row")

    for index, atleta in enumerate(atletas):
        try:
            place_overall = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-place.place-secondary.hidden-xs.numeric").text
        except:
            place_overall = "Sin posición general"

        try:
            place_gender = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-place.place-primary.numeric").text
        except:
            place_gender = "Sin posición género"

        try:
            nombre = atleta.find_element(By.CLASS_NAME, "list-field.type-fullname").text
        except:
            nombre = "Sin nombre"

        try:
            finish_net_label = atleta.find_element(By.XPATH, ".//div[.='Finish Net']")
            tiempo_neto = finish_net_label.find_element(By.XPATH, "../following-sibling::div").text.strip()
        except:
            tiempo_neto = "Sin tiempo neto"

        # Entrar al perfil individual para extraer división
        try:
            enlace = atleta.find_element(By.TAG_NAME, "a").get_attribute("href")
            driver.execute_script("window.open(arguments[0]);", enlace)
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(1.5)

            try:
                division = driver.find_element(By.XPATH, "//td[contains(@class, 'f-_type_age_class')]").text.strip()
            except:
                division = "Sin división"

            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        except:
            division = "Sin división"

        resultados.append({
            "page": pagina,
            "place_overall": place_overall,
            "place_gender": place_gender,
            "nombre": nombre,
            "tiempo_neto": tiempo_neto,
            "division": division
        })

        time.sleep(0.3)  # Breve pausa para estabilidad

    time.sleep(3)  # Pausa entre páginas

driver.quit()

